<a href="https://colab.research.google.com/github/AycaCicek/Turkcell-ML-DogrusalOlmayanRegresyonModelleri/blob/main/KKN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import scale,StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import neighbors
from sklearn.svm import SVR

In [2]:
data=pd.read_csv("/content/Hitters.csv")
data.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [3]:
from warnings import filterwarnings
filterwarnings("ignore")

# KNN

In [4]:
data=data.dropna()
data_encoding=pd.get_dummies(data[["League","Division","NewLeague"]])

#Bağımlı değişken
y = data["Salary"]
##BAĞIMSIZ DEĞİŞKEN SEÇİMİ##
X_=data.drop(["Salary","League","Division","NewLeague"],axis=1).astype("float64")
X=pd.concat([X_,data_encoding[["League_N","Division_W","NewLeague_N"]]],axis=1)

In [5]:
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


In [6]:
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.25,random_state=42)

Model & Tahmin

In [7]:
knn_model=KNeighborsRegressor().fit(X_train,y_train)

In [11]:
knn_model.n_neighbors

5

In [12]:
knn_model.metric

'minkowski'

In [13]:
dir(knn_model)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_algorithm_metric',
 '_check_feature_names',
 '_check_n_features',
 '_estimator_type',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_param_names',
 '_get_tags',
 '_kneighbors_reduce_func',
 '_more_tags',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_tree',
 '_validate_data',
 '_y',
 'algorithm',
 'effective_metric_',
 'effective_metric_params_',
 'feature_names_in_',
 'fit',
 'get_params',
 'kneighbors',
 'kneighbors_graph',
 'leaf_size',
 'metric',
 'metric_params',
 'n_features_in_',
 'n_jobs',
 'n_neighbors',
 'n_s

In [16]:
knn_model.predict(X_test)[0:5]

array([ 510.3334,  808.3334,  772.5   ,  125.5   , 1005.    ])

In [17]:
y_pred=knn_model.predict(X_test)

In [18]:
np.sqrt(mean_squared_error(y_test,y_pred))##ilkel test hatamız

426.6570764525201

MODEL TUNING

In [24]:
RMSE=[]

for k in range(10):
  k = k+1
  knn_model=KNeighborsRegressor(n_neighbors=k).fit(X_train,y_train)
  y_pred=knn_model.predict(X_test)
  rmse=np.sqrt(mean_squared_error(y_test,y_pred))
  RMSE.append(rmse)

  print(f"""k: {k} --> k Değeri için RMSE :{rmse}""")

k: 1 --> k Değeri için RMSE :455.03925390751965
k: 2 --> k Değeri için RMSE :415.99629571490965
k: 3 --> k Değeri için RMSE :420.6765370082348
k: 4 --> k Değeri için RMSE :428.8564674588792
k: 5 --> k Değeri için RMSE :426.6570764525201
k: 6 --> k Değeri için RMSE :423.5071669008732
k: 7 --> k Değeri için RMSE :414.9361222421057
k: 8 --> k Değeri için RMSE :413.7094731463598
k: 9 --> k Değeri için RMSE :417.84419990871265
k: 10 --> k Değeri için RMSE :421.6252180741266


In [25]:
##GridSearchCV
#kullanacak olduğumuzmakine öğrenmesi algoritmalarında belirlemeye çalıştığımız hiperparametre değerleri için kullanılan fonksiyon

In [69]:
knn_params={'n_neighbors': np.arange(1,30,1)}

In [70]:
knn_params

{'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])}

In [76]:
knn=KNeighborsRegressor()

In [72]:
knn.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [73]:
knn_cv_model = GridSearchCV(knn,knn_params,cv=10).fit(X_train,y_train)

In [79]:
knn_cv_model.best_params_

{'n_neighbors': 8}

In [75]:
##final modeli

In [80]:
knn_tuned=KNeighborsRegressor(n_neighbors=knn_cv_model.best_params_["n_neighbors"]).fit(X_train,y_train)

In [81]:
y_pred=knn_tuned.predict(X_test)

In [83]:
np.sqrt(mean_squared_error(y_pred,y_test))

413.7094731463598